
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



## Automatic Face Image Annotation  ##

We start off the data preparation phase by pre-processing images in our dataset for the unmasked category.

We have a selection of facial images and we need to draw bounding boxes around them. We could do that manually using a tool such as [labelImg](https://github.com/tzutalin/labelImg) however that would be slow, tedious and unnecessary for our purposes.

Instead we will use a facial detection algorithm (Haar cascade) to automatically detect faces and generate an accompanying XML file for each image with the coordinates of the face. Now this method is far from foolproof as it only works properly with full frontal images of faces but its a good starting point.



### Import required libraries

In [1]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/annotate.py

#!pip install lxml
#!pip install opencv-python

!pip install wget

from annotate import *
from constants import * 

#from lxml import etree as ET
#import cv2
#import wget
#import numpy as np
#from PIL import Image, ImageFile

import os
import glob
from pathlib import Path
import shutil



  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=e0cefc0939df320405c03a95e7f6ad734b2b2bf1364a4b63999f2a254b1e1d50
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
--2020-06-10 16:14:01--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253 (3.2K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   3.18K  --.-KB/s    in 0s      

2020-06-10 16:14:01 (45.0 MB/s) - ‘constants.py’ saved [3253/3253]



### Prepare dataset ###

#### Connect to Google Drive

In [0]:
from google.colab import drive

drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Download faces dataset and extract to temp folder


In [2]:
os.chdir(DATASET_DIR_TMP1)
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_00.zip
!unzip -j 1k_faces_00.zip '*.jpg'
!rm 1k_faces_00.zip


--2020-06-10 16:14:07--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_00.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23867089 (23M) [application/zip]
Saving to: ‘1k_faces_00.zip’

1k_faces_00.zip     100%[===================>]  22.76M  31.7MB/s    in 0.7s    

2020-06-10 16:14:08 (31.7 MB/s) - ‘1k_faces_00.zip’ saved [23867089/23867089]

Archive:  1k_faces_00.zip
  inflating: 0004STET6P.jpg          
  inflating: 000N7AIAFT.jpg          
  inflating: 00H858UYSD.jpg          
  inflating: 00KPGHV40E.jpg          
  inflating: 00P2LUXJW3.jpg          
  inflating: 00PYA83V1P.jpg          
  inflating: 00SD82OK2A.jpg          
  inflating: 00TMYXA5DF.jpg          
  inflating: 00V5CZZSSO.jpg          
  inflating

#### Download required dependency 

In [3]:
%cd /content/
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')

/content
--2020-06-10 16:03:39--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54039 (53K) [text/plain]
Saving to: ‘lbpcascade_frontalface_improved.xml’

lbpcascade_frontalf 100%[===================>]  52.77K  --.-KB/s    in 0.008s  

2020-06-10 16:03:40 (6.22 MB/s) - ‘lbpcascade_frontalface_improved.xml’ saved [54039/54039]



In [3]:

annotate(DATASET_DIR_TMP1,DATASET_DIR_TMP2)


--2020-06-10 16:14:15--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/annotate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2658 (2.6K) [text/plain]
Saving to: ‘annotate.py’

annotate.py         100%[===================>]   2.60K  --.-KB/s    in 0s      

2020-06-10 16:14:16 (49.1 MB/s) - ‘annotate.py’ saved [2658/2658]



In [0]:
for subdir, dirs, files in os.walk(DATASET_DIR_TMP1):
    for file in files:

        
        img_path=os.path.join(subdir, file)
        img_name=os.path.basename(img_path)

        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):

            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,w,bpp = np.shape(img)
            imgp = Image.open(img_path)
#            b, landmarks = detect_faces(imgp)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)


#            g=show_bboxes(imgp, b, landmarks)
            #print(len(faces))
            if len(faces) == 1 :
                try :

                    for bounding_boxes in faces:
                        face = img[int(bounding_boxes[1]):int(bounding_boxes[3]),
                        int(bounding_boxes[0]):int(bounding_boxes[2])]

                        
                        subdir_path, subdir_name = os.path.split(subdir)

                        root = ET.Element("annotation",verified="yes")
                        ET.SubElement(root, "folder").text=DATASET_DIR_TMP1

                        ET.SubElement(root, "filename").text = img_name
                        #ET.SubElement(root, "path").text = img_path

                        source=ET.SubElement(root, "source")
                        ET.SubElement(source, "database").text = "Cloud Commander"

                        size=ET.SubElement(root, "size")
                        ET.SubElement(size, "width").text = str(w)
                        ET.SubElement(size, "height").text = str(h)
                        ET.SubElement(size, "depth").text = str(bpp)

                        ET.SubElement(root, "segmented").text = "0"

                        obj=ET.SubElement(root, "object")
                        ET.SubElement(obj, "name").text = subdir_name
                        ET.SubElement(obj, "pose").text = "Frontal"
                        ET.SubElement(obj, "truncated").text = "0"
                        ET.SubElement(obj, "difficult").text = "0"

                        box=ET.SubElement(obj, "bndbox")
                        ET.SubElement(box, "xmin").text = str(int(bounding_boxes[0]))
                        ET.SubElement(box, "ymin").text = str(int(bounding_boxes[1]))
                        ET.SubElement(box, "xmax").text = str(int(bounding_boxes[2])+int(bounding_boxes[1]))
                        ET.SubElement(box, "ymax").text = str(int(bounding_boxes[3]))

                        tree = ET.ElementTree(root)
                        tree.write(os.path.join(DATASET_DIR_TMP2, os.path.splitext(img_name)[0] + '.xml'))
                        
                        
                except RuntimeError :
                    with open("delete.txt", "a") as myfile:
                        myfile.write(img_path+"\n")
                	

                else :
                    with open("delete.txt", "a") as myfile:
                            myfile.write(img_path+"\n")
                    

### Save Dataset

In [0]:
def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        if f.endswith(('.png', '.jpg', '.jpeg', '.xml')):
          shutil.move(source+"/"+f, destination+"/")


move_files(DATASET_DIR_TMP1,DATASET_DIR_PREP_TRAIN_IMG)
move_files(DATASET_DIR_TMP2,DATASET_DIR_PREP_TRAIN_ANNO)


In [5]:
!ls /content/tensorflow/workspace/face-mask/dataset/prepared/train/annotations/

0004STET6P.xml	0WRX6JWHIV.xml	1W2BZZF9EJ.xml	2W6L6NNTWZ.xml	3TI542YWBS.xml
000N7AIAFT.xml	0WXKO4TSI6.xml	1WIKLJOI5D.xml	2W7WGFHA79.xml	3TJ2GA82G2.xml
00H858UYSD.xml	0X720RKIMQ.xml	1WN5PESA47.xml	2W8WZ3XVFE.xml	3TJLT955O4.xml
00KPGHV40E.xml	0X7Q6KSSD8.xml	1WPU9H0C7M.xml	2WA7O2SG0J.xml	3TYLBBCF7P.xml
00P2LUXJW3.xml	0XB41CUJRQ.xml	1WSXZSOG22.xml	2WOG6CT55I.xml	3U5NHFGLX7.xml
00PYA83V1P.xml	0XG0O1EPVC.xml	1WWDZZQ61G.xml	2WOQ223DVN.xml	3U6FW61UTL.xml
00SD82OK2A.xml	0XGXMB3D0Y.xml	1X5JZFZN0U.xml	2WP3A4TFXW.xml	3U6YDZ3ME8.xml
00TMYXA5DF.xml	0XPRPD81KY.xml	1X8BIWF3ZK.xml	2WSQQA9EQB.xml	3UTVCDIP5O.xml
00V5CZZSSO.xml	0YCJK7YC1R.xml	1XBSB6ENR1.xml	2WUX27JV6W.xml	3UUYMGSKJ5.xml
00W5NPIX4S.xml	0YFK3LM2AZ.xml	1XE5JFM5D0.xml	2WYJAOGZ3U.xml	3UV7UI3MFJ.xml
00XN46VW5G.xml	0YMO27R9LN.xml	1XES48EAIG.xml	2X1YP8855F.xml	3UWU6XZQZV.xml
01D47ZIGE0.xml	0YWWFJCKLA.xml	1XLROGKRLU.xml	2X2N94YW6D.xml	3UZJ3QZON1.xml
01KDGEL6IQ.xml	0YXNAIUF1Y.xml	1XNO6CO064.xml	2X3FWOCSDW.xml	3V4EDIYT96.xml
01NH00JKMJ.xml	0Z2RSIN1T8

### Compress Dataset (Optional) ###

Follow this step if you wish to archive your dataset for future use

In [0]:
!zip -r unmasked-dataset.zip /content/training_demo/images

  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/2OKEGQH2XR.jpg (deflated 0%)
  adding: content/training_demo/images/3QOQ2TMK9U.jpg (deflated 0%)
  adding: content/training_demo/images/0G8BKB200W.jpg (deflated 0%)
  adding: content/training_demo/images/4B5AC7FB7W.jpg (deflated 0%)
  adding: content/training_demo/images/0NZVBZ17MN.jpg (deflated 0%)
  adding: content/training_demo/images/2MAKY44FDM.jpg (deflated 0%)
  adding: content/training_demo/images/0TNTAUJVU8.xml (deflated 47%)
  adding: content/training_demo/images/3U6YDZ3ME8.xml (deflated 47%)
  adding: content/training_demo/images/3F8M5W3XFL.xml (deflated 47%)
  adding: content/training_demo/images/4CNDRH3OX7.xml (deflated 47%)
  adding: content/training_demo/images/1GSOH70PZB.xml (deflated 47%)
  adding: content/training_demo/images/00XN46VW5G.jpg (deflated 0%)
  adding: content/training_demo/images/2TAJDIE93R.xml (deflated 47%)
  adding: content/training_demo/images/1TJ1CYQWP7.jpg (def

#### Save archive to Google Drive


In [0]:
!gsutil cp unmasked-dataset.zip /content/drive/My\ Drive/Datasets/

Copying file://unmasked-dataset.zip...
/ [1 files][ 23.2 MiB/ 23.2 MiB]                                                
Operation completed over 1 objects/23.2 MiB.                                     
